In [1]:
%matplotlib inline

In [2]:
DATA_HOME_DIR = "data/dogscats"

path = DATA_HOME_DIR + '/' #'/sample/'
# path = DATA_HOME_DIR + '/sample/'

test_path = DATA_HOME_DIR + '/test1/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'


In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
from importlib import reload
import utils; reload(utils)
from utils import plots

Using Theano backend.


In [5]:
from vgg16 import Vgg16
batch_size =16
vgg = Vgg16()

In [6]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
vgg.finetune(batches)

In [8]:
USE_SAVED_MODEL = True

In [9]:
no_of_epochs = 10
latest_weights_filename = None
if not USE_SAVED_MODEL:
    for epoch in range(no_of_epochs):
        print("Running epoch: %d" % epoch)
        vgg.fit(batches, val_batches, nb_epoch=1)
        latest_weights_filename = 'ft%d.h5' % epoch
        vgg.model.save_weights(results_path+latest_weights_filename)

In [10]:
if USE_SAVED_MODEL:
    vgg.model.load_weights(results_path+'ft7.h5')

In [11]:
batches, preds = vgg.test(path+'test1', batch_size = batch_size*2)

Found 12500 images belonging to 1 classes.


In [18]:
import pandas as pd
import numpy as np

In [19]:
df=pd.DataFrame(preds, columns=["isCat","isDog"])
df['filename'] = batches.filenames
df['id'] = df['filename'].apply(lambda x: int(x.replace('unknow/', '').replace('.jpg', '')))
df['label'] = np.clip(df['isDog'], 0.05, 0.95)
df = df.set_index(['id'])
df = df.sort_index()

In [21]:
df.to_csv('catvsdog_sub.csv', columns=['label'])